In [2]:
! wget https://storage.googleapis.com/kaggle-forum-message-attachments/237294/7771/german_credit_data.csv

--2020-05-24 14:14:43--  https://storage.googleapis.com/kaggle-forum-message-attachments/237294/7771/german_credit_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.1.48
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.1.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53393 (52K) [text/csv]
Saving to: ‘german_credit_data.csv’

german_credit_data. 100%[===================>]  52.14K  --.-KB/s    in 0.04s   

2020-05-24 14:14:44 (1.19 MB/s) - ‘german_credit_data.csv’ saved [53393/53393]



In [3]:
import pandas as pd
df = pd.read_csv("german_credit_data.csv", index_col=0)

In [4]:
df = df.fillna(value="NA")

In [5]:
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NA,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NA,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NA,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NA,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
preprocess = make_column_transformer(
    (StandardScaler(), ['Age', 'Credit amount', 'Duration']),
    (OneHotEncoder(sparse=False), ['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk'])
)

In [9]:
mat = preprocess.fit_transform(df)

In [10]:
mat[10,:]

array([-0.92754658, -0.70047155, -0.73866754,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ])

In [11]:
X = mat[:, :-2]
Y = mat[:, -1]

In [12]:
num_feats = X.shape[1]
numX = X.shape[0]

In [13]:
import numpy as np

In [14]:
datasize = 500
cs_size = 25
split_on_doc = 0.8
testsize = 100
ratio_relevant = 0.4
ratios_col = Y * ratio_relevant + (1-Y)*(1-ratio_relevant)

In [15]:
# generate a candidate set of size 10 everytime
data_X = []
data_Y = []
test_X = []
test_Y = []
group_identities_train = []
group_identities_test = []
print("Sampling between 0 and {} for train".format(numX*split_on_doc))
p = ratios_col[0:int(numX*split_on_doc)]
p = p / sum(p)
for i in range(datasize):
    cs_indices = np.random.choice(np.arange(0, numX*split_on_doc, dtype=int), size=cs_size, p=p)
    cs_X = X[cs_indices]
    cs_Y = Y[cs_indices]
    data_X.append(cs_X)
    data_Y.append(cs_Y)
    group_identities_train.append(cs_X[:,4])
print("Sampling between {} and {} for test".format(numX*split_on_doc, numX))
p = ratios_col[int(numX*split_on_doc):]
p = p/sum(p)
for i in range(testsize):
    cs_indices = np.random.choice(np.arange(numX*split_on_doc, numX, dtype=int), size=cs_size, p=p)
    test_X.append(X[cs_indices])
    test_Y.append(Y[cs_indices])
    group_identities_test.append(X[cs_indices,4])

Sampling between 0 and 800.0 for train
Sampling between 800.0 and 1000 for test


In [17]:
import pickle as pkl
pkl.dump((data_X, data_Y), open("../../german/their_german_train_rank.pkl", "wb"))
pkl.dump((test_X, test_Y), open("../../german/their_german_test_rank.pkl", "wb"))


In [26]:
X_train = np.zeros((cs_size*datasize, data_X[0].shape[1]))
relevance_train = np.zeros(cs_size*datasize)
for i in range(len(data_X)):
    X_train[cs_size*i:(i+1)*cs_size,:] = data_X[i]
    relevance_train[cs_size*i:(i+1)*cs_size] = data_Y[i]

In [27]:
X_test = np.zeros((cs_size*testsize, data_X[0].shape[1]))
relevance_test = np.zeros(cs_size*testsize)
for i in range(len(test_X)):
    X_test[cs_size*i:(i+1)*cs_size,:] = test_X[i]
    relevance_test[cs_size*i:(i+1)*cs_size] = test_Y[i]

In [33]:
data_X

KeyboardInterrupt: 

In [34]:
pkl.dump((X_train, relevance_train, X_train[:,3]), open("../../german/german_train_rank.pkl", "wb"))
pkl.dump((X_test, relevance_test, X_test[:,3]), open("../../german/german_test_rank.pkl", "wb"))